In [1]:
pip install pymysql

   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install SQLAlchemy

In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas==1.3.5

  Using cached pandas-1.3.5.tar.gz (4.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pandas
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [2474 lines of output]
  <string>:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\_typing.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-311\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-311\pandas
  creating build\lib.win-amd64-cpython-311\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-311\pandas\api
  creating build\lib.win-amd64-cpython-311\pandas\arrays
  copying pandas\arrays\__init__.py -> build\lib.win-amd64-cpython-311\p

In [1]:
import pandas
print(pandas.__version__)

2.1.4


In [4]:
pip install sqlalchemy==1.4.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlalchemy
print(sqlalchemy.__version__)

1.4.0


In [3]:
import pymysql
from sqlalchemy import create_engine

# 데이터베이스 설정
db_user = 'root'
db_pass = '43qjsrnfjddl!@'
db_host = 'localhost'
db_name = 'sp500_db'
engine = create_engine(f'mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}')

try:
    connection = engine.connect()
    print("연결 성공!")
    connection.close()
except Exception as e:
    print("연결 실패:", e)

연결 성공!


In [2]:
from sqlalchemy import create_engine
import pandas as pd
import yfinance as yf
import requests
import pymysql


# 데이터베이스 설정
db_user = 'root'
db_pass = '43qjsrnfjddl!@'
db_host = 'localhost'
db_name = 'sp500_db'
engine = create_engine(f'mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}')
'''
mysql+pymysql://: 이 부분은 데이터베이스와의 연결에 사용될 DBAPI의 종류를 나타냅니다. 여기서 mysql은 사용할 데이터베이스의 종류를 나타내고, 
pymysql은 MySQL 데이터베이스에 연결하기 위해 사용되는 Python DBAPI입니다. 
SQLAlchemy는 이 정보를 사용하여 적절한 데이터베이스 드라이버를 선택하고 연결합니다.

{db_user}: 데이터베이스에 로그인할 때 사용되는 사용자 이름입니다.

{db_pass}: 해당 사용자의 비밀번호입니다.

@{db_host}: 데이터베이스 서버의 호스트 이름이나 IP 주소입니다. localhost를 사용하면 현재 머신에 설치된 데이터베이스 서버에 연결하려고 시도합니다. 원격 데이터베이스 서버에 연결하려면, 해당 서버의 IP 주소나 도메인 이름을 제공해야 합니다.

/{db_name}: 연결하려는 데이터베이스의 이름입니다.
'''
try:
    connection = engine.connect()
    print("연결 성공!")
    connection.close()
except Exception as e:
    print("연결 실패:", e)

def save_companies_data():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    selected_info = []

    for ticker_symbol in sp500['Symbol']:
        stock = yf.Ticker(ticker_symbol)
        info = stock.info
        selected_info.append({
            "symbol": info.get("symbol"),
            "companyName": info.get("longName"),
            "industry": info.get("industry"),
            "sector": info.get("sector"),
        })

    data_df = pd.DataFrame(selected_info)
    data_df.to_sql('companies', engine, if_exists='append', index=False)

def save_stock_prices():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    data = []

    for ticker_symbol in sp500['Symbol']:
        stock = yf.Ticker(ticker_symbol)
        info = stock.info
        data.append({
            'Symbol': ticker_symbol,
            'Name': info.get('shortName'),
            'Sector': info.get('sector'),
            'Country': info.get('country')
        })

    data_df = pd.DataFrame(data)
    data_df.to_sql('stock_prices', engine, if_exists='append', index=False)

def fetch_news(api_key, symbol, company_name, from_date, to_date, language='en'):
    query = f'"{symbol}" OR "{company_name}"'
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,
        'from': from_date,
        'to': to_date,
        'language': language,
        'sortBy': 'publishedAt',
        'apiKey': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        articles = response.json()['articles']
        articles_data = [{
            'Symbol': symbol,
            'Company': company_name,
            'Title': article['title'],
            'Date': article['publishedAt'][:10],  # 날짜만 저장
            'URL': article['url']
        } for article in articles]

        articles_df = pd.DataFrame(articles_data)
        articles_df.to_sql('news_articles', engine, if_exists='append', index=False)

def save_all_news():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    api_key = 'aa385e54196f4d278833fd4f8b663b1e'
    from_date = '2024-02-01'
    to_date = '2024-02-29'

    for index, row in sp500.iterrows():
        fetch_news(api_key, row['Symbol'], row['Company'], from_date, to_date)

# 기업 데이터 저장 함수
def save_enterprise_data():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    enterprise_info = []

    for ticker_symbol in sp500['Symbol']:
        stock = yf.Ticker(ticker_symbol)
        info = stock.info
        enterprise_info.append({
            "symbol": info.get("symbol"),
            "companyName": info.get("longName"),
            "industry": info.get("industry"),
            "sector": info.get("sector"),
            "netIncomeToCommon": info.get("netIncomeToCommon", 0),
            "totalRevenue": info.get("totalRevenue", 0),
            "ebitda": info.get("ebitda", 0),
            "totalDebt": info.get("totalDebt", 0),
            "currentRatio": info.get("currentRatio", 0.0),
            "returnOnAssets": info.get("returnOnAssets", 0.0),
            "returnOnEquity": info.get("returnOnEquity", 0.0),
            "grossMargins": info.get("grossMargins", 0.0),
            "operatingMargins": info.get("operatingMargins", 0.0),
        })

    enterprise_df = pd.DataFrame(enterprise_info)
    # 'if_exists' 매개변수를 'append'로 설정하여 기존 데이터에 추가합니다.
    enterprise_df.to_sql('enterprise_data', engine, if_exists='append', index=False)
    print("Enterprise data saved to database.")

# 함수 호출
save_companies_data()
save_stock_prices()
save_all_news()
save_enterprise_data()

# 주의: `save_enterprise_data` 함수는 예시로 제공된 정보를 기반으로 작성되어야 하며, 여기서는 구현되지 않았습니다. 실제 구현에는 `yfinance`를 사용하여 필요한 기업 데이터를 가져오고, 이를 `enterprise_data` 테이블에 저장하는 로직이 포함되어야 합니다.


연결 실패: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '@localhost' ([Errno 11003] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '@localhost' ([Errno 11003] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [3]:
import pandas as pd
import yfinance as yf
import pandas as pd
import yfinance as yf
from sqlalchemy import create_engine

# 데이터베이스 설정
db_user = 'root'
db_pass = '43qjsrnfjddl!@'
db_host = 'localhost'
db_name = 'sp500_db'
engine = create_engine(f'mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}')


def save_companies_data():
    sp500 = pd.read_csv('./SP500_Companies.csv')
    sp500_tickers = sp500['Symbol']

    selected_info = []
    for ticker_symbol in sp500_tickers:
        stock = yf.Ticker(ticker_symbol)
        info = stock.info
        selected_info.append({
            "symbol": info.get("symbol"),
            "companyName": info.get("longName"),
            "industry": info.get("industry"),
            "sector": info.get("sector"),
        })

    data_df = pd.DataFrame(selected_info)
    data_df.to_sql('companies', engine, if_exists='append', index=False)
    print("Companies data saved to database.")

In [ ]:
import requests
from sqlalchemy import create_engine

# 데이터베이스 설정
db_user = 'root'
db_pass = '43qjsrnfjddl!@'
db_host = 'localhost'
db_name = 'sp500_db'
engine = create_engine(f'mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}')

def fetch_and_save_news(api_key, symbol, company_name):
    query = f'"{symbol}" OR "{company_name}"'
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,
        'from': '2024-02-01',
        'to': '2024-02-29',
        'sortBy': 'publishedAt',
        'apiKey': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        articles = response.json()['articles']
        articles_data = [{
            'Symbol': symbol,
            'Company': company_name,
            'Title': article['title'],
            'Date': article['publishedAt'],
            'URL': article['url']
        } for article in articles]

        articles_df = pd.DataFrame(articles_data)
        articles_df.to_sql('news_articles', engine, if_exists='append', index=False)
        print(f"News articles for {company_name} saved to database.")

try:
    connection = engine.connect()
    print("연결 성공!")
    connection.close()
except Exception as e:
    print("연결 실패:", e)
